In [ ]:
import numpy as np
from pathlib import Path
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.applications import VGG16
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from PIL import Image

In [ ]:
# Paths and configuration
DATA_DIR = Path("/Users/bradrichardson/alzheimers-detection/data")
MODEL_SAVE_PATH = Path("models/alzheimer_model.h5")
HISTORY_SAVE_PATH = Path("models/training_history.npy")
CLASSES = ["non_demented", "very_mild_dementia", "mild_dementia", "moderate_dementia"]
IMAGE_SIZE = (128, 128)

In [ ]:
# Class weights
class_weights = {
    0: 0.32,  # non_demented
    1: 1.57,  # very_mild_dementia
    2: 4.32,  # mild_dementia
    3: 44.25  # moderate_dementia
}

In [ ]:
# Load and preprocess data
def load_data(data_dir, classes, image_size):
    data = []
    labels = []
    encoder = OneHotEncoder()
    encoder.fit(np.array(range(len(classes))).reshape(-1, 1))
    
    for i, cls in enumerate(classes):
        class_dir = data_dir / cls
        for img_path in class_dir.glob("*.jpg"):
            try:
                img = Image.open(img_path).resize(image_size)
                img_array = np.array(img)
                if img_array.shape == (*image_size, 3):  # Ensure consistent image shape
                    data.append(img_array)
                    labels.append(encoder.transform([[i]]).toarray().flatten())
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
    return np.array(data), np.array(labels)

In [ ]:
# Load dataset
data, result = load_data(DATA_DIR, CLASSES, IMAGE_SIZE)

In [ ]:
# Split into training, validation, and test sets
x_train, x_test, y_train, y_test = train_test_split(data, result, test_size=0.15, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.15, random_state=42)

In [ ]:
# Normalize data
x_train, x_val, x_test = x_train / 255.0, x_val / 255.0, x_test / 255.0

In [ ]:
# Build the model using a pre-trained VGG16
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(128, 128, 3))
for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation="relu"),
    Dropout(0.5),
    Dense(4, activation="softmax")
])


In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# Training with early stopping
early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=20,
    batch_size=32,
    class_weight=class_weights,
    callbacks=[early_stopping]
)

In [ ]:
# Save model and training history
model.save(MODEL_SAVE_PATH)
np.save(HISTORY_SAVE_PATH, history.history)